In [44]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image, ImageDraw
import requests
from pycocotools.coco import COCO
from pathlib import Path
import cv2
import torch
import os
import sys
from torchvision import transforms
import mrcnn

In [45]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './Mask_RCNN'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
# import mrcnn.utils as utils
from mrcnn import visualize
# import mrcnn.model as modellib
from torchvision.ops import box_convert

In [46]:
# def display_image(img):
#     if isinstance(img, type(np.array)):
#          display(Image.fromarray(img))
#     else:
#         display(Image.fromarray(cv2.imread(img, cv2.IMREAD_COLOR))) 

In [47]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [68]:
class LiveCellDataset(torch.utils.data.Dataset): #torch.utils.data.Dataset
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        
        # Loading annotations here
        self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_train_trashcan.json")
        
        # Loading their ids, please note only one category!
        self.image_ids = self.coco_annotation.getImgIds(catIds=[self.coco_annotation.getCatIds()[0]])

    def __getitem__(self, idx):
        # loading img:
        img_id = self.image_ids[idx]
        img_info = self.coco_annotation.loadImgs([img_id])[0]
        img_file_name = img_info["file_name"]
        
        splitted = img_file_name.split("_")
        img_path = Path("../data/LiveCell/images/livecell_test_images/") / splitted[0] / img_file_name
        #img_path = Path("../data/TrashCan/train") / img_file_name
        
        img = Image.open(str(img_path)).convert("RGB")
        img = np.asarray(img)
        
#         display_image(img)
        
        ann_ids = self.coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
        anns = self.coco_annotation.loadAnns(ann_ids)
        
        masks, class_ids = self.get_masks(anns, img_info)
#         print(masks)
        
        boxes = [a_dict['bbox'] for a_dict in anns]
        iscrowd = [a_dict['iscrowd'] for a_dict in anns]
        labels = [a_dict['category_id'] for a_dict in anns]
        area = [a_dict['area'] for a_dict in anns]
        
# 
        
        
#         num_objs = len(class_ids)
#         boxes = []
#         for i in range(num_objs):
# #             print(masks[i,:,:])
#             pos = np.where(masks[:,:,i])
#             xmin = np.min(pos[1])
#             xmax = np.max(pos[1])
#             ymin = np.min(pos[0])
#             ymax = np.max(pos[0])
#             boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
    

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        boxes = box_convert(boxes, in_fmt='xywh', out_fmt='xyxy')
        assert len(boxes) == len(iscrowd) == len(labels) == len(area)
        if not (len(boxes) == len(iscrowd) == len(labels) == len(area)):
            print(len(boxes), len(iscrowd), len(labels), len(area))
#         cat_names = self.coco_annotation.loadCats(class_ids)\
        #
#         cat_names = np.array([x['name'] for x in cat_names])
#         list_of_cat_names = [cat["name"] for cat in cats]

#         cat_names = []
#         for class_id in class_ids:
#             cat_names.append(list_of_cat_names[class_id+1])
        cat_names = np.array(['cell' for x in range(0, len(class_ids))])
        masks = torch.as_tensor(masks, dtype=torch.uint8)
#         boxes = torch.as_tensor(boxes, dtype=torch.float32)
#         boxes = box_convert(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor(area, dtype=torch.float32) #Should 64 be here?
#         img = torch.as_tensor(img, dtype=torch.float64)
        image_id = torch.tensor([idx])
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        target["class_ids"] = class_ids
        target["cat_names"] = cat_names

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        return transforms.ToTensor()(img), target
#         return img, target
    
    def get_masks(self, annotations, image_info):
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids
        

    def __len__(self):
        return len(self.image_ids)

In [69]:
dataset = LiveCellDataset("LiveCellDataset", None)

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [74]:
image, target = dataset.__getitem__(12)
target

{'boxes': tensor([[198.0020, 232.0020, 479.9980, 324.9980]]),
 'labels': tensor([1]),
 'masks': tensor([[[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
 
         [[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
 
         [[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
 
         ...,
 
         [[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
 
         [[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
 
         [[0, 0],
          [0, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]]], dtype=torch.uint8),
 'image_id': tensor([12]),
 'area': tensor([11412.7500]),
 'iscrowd': tensor([0]),
 'class_ids': array([1, 1], dty

In [71]:
target['area']

tensor([11412.7500])

In [72]:
image.shape

torch.Size([3, 360, 480])

In [73]:
image_ids = np.random.choice(list(range(0, dataset.__len__())), 4)
print(image_ids)
for image_id in image_ids:
    image, target = dataset.__getitem__(image_id)

    visualize.display_top_masks(image, target['masks'], target["class_ids"], target["cat_names"])

[ 148  412  974 1576]


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [38]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [39]:
data_loader = torch.utils.data.DataLoader(
dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [40]:
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
# print(targets.values())

In [41]:
import torchvision


In [42]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [43]:
import torchvision
from torchvision.references.engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    # let's train it for 10 epochs
    num_epochs = 1

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader, device=device)

    print("That's it!")
main()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.